# NOTA: MARCO CAMBIA GLI INDICI

In [ ]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

# Parametri default

In [ ]:
seed = None     # random seed
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [ ]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1,1,5,10]
problem_sizes = (200, 300, 400, 600)
n_prob = 10

In [ ]:
#feat, targ = load_ml_dataset()
#t1 = targ[:,0]
#t2 = targ[:,1]
#targ = t1

In [ ]:
solvers.options['show_progress'] = False
def solve_with_cvxopt(K, target, size, u, e):
    A = np.block([[K, -K], [-K, K]])
    b = np.block([e - target, e + target])
    a = np.block([np.ones(n), -np.ones(n)])
    _G, _A, _h, _b = build_problem(2*size, u)
    Q1 = matrix(A)
    q1 = matrix(b)
    G1 = matrix(_G)
    h1 = matrix(_h)
    A1 = matrix(_A)
    b1 = matrix(_b) 
    sol = solvers.qp(Q1, q1, G1, h1, A1, b1)
    return sol['primal objective']

In [ ]:
def generate_and_save(feat, targ, size, u, epsilon, n, startdir='CSV'):
    os.makedirs(f"{startdir}/{size}/GOLD", exist_ok=True)
    os.makedirs(f"{startdir}/{size}/ROS", exist_ok=True)
    with open(f"{startdir}/{size}/optimal_values.txt", "a") as file:
        for i in range(n):
            print(f"generating problem {i}")
            K, target = sample_transform_problem(feat, targ, size)
            print("problem generated\nsoving it with the solver")
            sol = solve_with_cvxopt(K, target, size, u)
            file.write(str(sol)+'\n')
            ds = DataStorer()
            print("About to run GOLDSTEIN")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print("About to run ROSEN")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}/{size}/ROS/{i}.csv")
        

In [ ]:
def generate_multiple_epsilons(feat, targ, size, u, epsilons, n, startdir='./'):
    u_name = str(u).replace(".","")
    for e in epsilons:
        eps_name = str(e).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for e in epsilons:
            print(f"Solving problem {i} with epsilon {e}")
            eps_name = str(e).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")

In [ ]:
def generate_multiple_upperbounds(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    
    eps_name = str(epsilon).replace(".", "")
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
        
    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for u in upper_bounds:
            print(f"Solving problem {i} with U: {u}")
            u_name = str(u).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")

# CALIFORNIA HOUSING

In [ ]:
feat, targ = load_california_dataset()
targ

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir="./CAL/E_")

In [ ]:
for size in problem_sizes:
generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir="./CAL/C_")

# AIRFOIL

In [ ]:
feat, targ = load_airfoil_dataset()
targ = targ.ravel()
targ

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir="./AIRF/E_")

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir="./AIRF/C_")